In [ ]:
# notebooks/inference.ipynb

import os
import pandas as pd
import torch
from torchvision import models
from torch.utils.data import DataLoader
from src.preprocessing import SoilDataset, get_transforms
from src.postprocessing import map_predictions

# Paths
test_dir = "/kaggle/input/soil-classification/soil_classification-2025/test/"
test_ids_csv = "/kaggle/input/soil-classification/soil_classification-2025/test_ids.csv"
model_path = "/kaggle/working/soil_resnet18.pth"
submission_path = "/kaggle/working/submission.csv"

# Load model
model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, 4)
model.load_state_dict(torch.load(model_path, map_location="cpu"))
model = model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

# Prepare test data
test_ids = pd.read_csv(test_ids_csv)
transform = get_transforms(train=False)
test_dataset = SoilDataset(test_dir, test_ids, transform=transform, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Inference
predictions = []
with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs.to("cuda")
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy())

# Map and save submission
submission_df = test_ids.copy()
submission_df['label'] = map_predictions(predictions)
submission_df.to_csv(submission_path, index=False)
print("Submission saved at", submission_path)
